<a href="https://colab.research.google.com/github/KSY1526/myblog/blob/master/_notebooks/2021-12-22-Do_natural_language3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do it 자연어) 3. 문서에 꼬리표 달기 + 웹 실습


# 모델 환경설정

In [ ]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 2.8 MB 10.8 MB/s 
     |████████████████████████████████| 806 kB 32.2 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 829 kB 35.1 MB/s 
     |████████████████████████████████| 332 kB 42.8 MB/s 
     |████████████████████████████████| 132 kB 46.9 MB/s 
     |████████████████████████████████| 636 kB 45.5 MB/s 
     |████████████████████████████████| 61 kB 504 kB/s 
     |████████████████████████████████| 3.3 MB 31.9 MB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 192 kB 46.4 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 160 kB 36.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from torch.cuda import is_available
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name='beomi/kcbert-base', # 프리트레인 마친 언어모델의 이름
    downstream_corpus_name='nsmc', # 다운스트림 데이터 이름
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', # 파인튜닝된 모델의 체크포인트가 저장될 위치.
    batch_size = 32 if torch.cuda.is_available() else 4, #배치 크기.
    learning_rate=5e-5,
    max_seq_length = 128, # 토큰 기준 입력 문장 최대 길이
    epochs = 1, # 학습 데이터 3번 반복
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed = 7,
)

kcbert-base 모델을 NSMC 데이터로 파인튜닝 합니다.

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


랜덤 시드를 설정합니다.

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


각종 로그를 출력하는 로거를 설정합니다.

# 말뭉치 내려받기

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name, # 다운스트림 데이터 이름
    root_dir = args.downstream_corpus_root_dir, # 다운스트림 데이터를 내려받을 위치.
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 69.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.8MB/s]                            


네이버 영화리뷰 데이터(NSMC)를 내려받습니다.

Korpora를 이용해 말뭉치를 특정장소에 저장합니다.

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

토큰화를 수행하는 토크나이저를 선언합니다.

# 학습 데이터 구축

In [ ]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset

corpus = NsmcCorpus() # csv 파일형식의 NSMC 데이터를 문장과 레이블(긍/부정)으로 읽어 들임.
train_dataset = ClassificationDataset(
    args = args,
    corpus = corpus, 
    tokenizer = tokenizer,
    mode = 'train',
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /root/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 45.759 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

학습 데이터 셋을 구축했습니다.

여기서 ClassificationDataset은 NsmcCorpus가 넘겨준 문장과 레이블을 각각 tokenizer을 활용해 가공합니다.

이때 가공된 형태는 ClassificationFeatures 자료형으로 4가지 정보가 있습니다.

첫번째로 input_ids로 토큰 시퀀스를 의미합니다. 두번째는 attention_mask로 패딩 토큰(0)을 구분합니다.

세번째는 token_type_ids로 세그먼트 정보, 네번째는 label로 레이블 정보 입니다.

In [ ]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_dataset[100]

ClassificationFeatures(input_ids=[2, 2005, 4024, 4017, 1293, 4599, 4775, 4042, 2478, 4075, 4196, 15, 1463, 4207, 4196, 8080, 4024, 10314, 11219, 4180, 12610, 10579, 832, 4140, 11414, 9827, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

ClassificationFeatures 자료형을 개별 자료마다 확인했습니다. 4가지 정보가 있는걸 확인 할 수 있어요.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False), # batch_size만큼 비복원 랜덤 추출.
    collate_fn = nlpbook.data_collator, # 뽑은 인스턴스를 배치로 만드는 역할. 
    # 인스턴스들을 앞서 말한 4가지 정보별로 취합 후 파이토치가 요구하는 자료형인 텐서 형태로 바꿈.
    drop_last = False,
    num_workers = args.cpu_workers,
)

학습 데이터 셋으로부터 로더를 구축했습니다.

배치 크기만큼 인스턴스를 뽑아 이를 합처서 배치를 만듭니다.

# 평가용 데이터 구축

In [ ]:
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode='test',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /root/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 29.230 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

평가용 데이터는 인스턴스를 랜덤하게 뽑을 필요가 없습니다.

그러므로 SequentialSampler을 사용해 인스턴스를 순서대로 추출합니다.

# 모델 학습

In [ ]:
args.pretrained_model_name

'beomi/kcbert-base'

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name, # 프리트레인을 마친 BERT인 beomi/kcbert-base를 사용한다.
    num_labels = corpus.num_labels, # 긍정/부정을 분류하므로 2이다.
)

model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

BertForSequenceClassification 모델은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태입니다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationTask

task = ClassificationTask(model, args)

ClassificationTask에는 옵티마이저(모델 파라미터를 업데이트 할때 최적의 방향/보폭 설정하는 도구), learning rata 스케줄러가 정의됩니다.

In [ ]:
trainer = nlpbook.get_trainer(args)
trainer

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


트레이너를 정의했습니다. 트레이너는 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정을 알아서 해줍니다.

In [ ]:
trainer.fit(
    task,
    train_dataloader = train_dataloader,
    val_dataloaders = val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1 에포크당 약 2시간?이 걸린다..

# 영화 리뷰 감성분석 웹 서비스 만들기

In [ ]:
# !pip install ratsnlp

# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

# from transformers import BertConfig, BertForSequenceClassification
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained(
#     args.pretrained_model_name,
#     do_lower_case = False
# )

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', 
    max_seq_length = 128, 
)

인퍼런스 설정을 해줍니다. 프리트레인 모델은 앞선 분석에서 쓴 모델과 같은 모델을 사용해야합니다.

In [ ]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu'),
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name, 
    num_labels = 2
)

model = BertForSequenceClassification.from_pretrained(pretrained_model_config)

AttributeError: ignored

앞서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

그 뒤 BERT 모델을 초기화 해줍니다.

In [ ]:
model.load_state_dict({k.replace('model.',''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

초기화한 BERT 모델에 체크포인트를 주입합니다.

그 이후 모델을 평가모드로 전환합니다. 드롭아웃 등 학습 때만 사용하는 기능을 무력화 시키기 위해서 입니다.

(체크포인트란 모델이 사용하는 모든 매개변수에 대한 정확한 값을 기록하는 것)

In [ ]:
from numpy.core.numeric import outer
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length = args.max_seq_length,
        padding = 'max_length',
        truncation = True,
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) # items : 키와 값 둘다 얻는 함수.
        prob = outputs.logits.softmax(dim = 1) # 로짓에 소프트맥스 취하기
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = '긍정' if torch.argmax(prob) == 1 else '부정'

    return {
        'sentence' : sentence,
        'prediction' : pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

문장을 입력받으면 모델을 사용해 긍정/부정을 판단해주는 함수를 만듭니다.

모델은 로짓값을 출력하므로 소프트맥스를 이용해 확률 값으로 나타냅니다.

 with torch.no_grad()은 오차 역전파에 사용하는 계산량을 줄여서 처리 속도를 높입니다.

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

플라스크라는 파이썬 라이브러리의 도움을 받아 웹 서비스를 할 수 있습니다.

(딥러닝 모델에서 약 6시간.. 너무 오래 걸려서 오류 그대로 업로드 합니다.)